In [25]:
import pandas as pd

import numpy as np
import tensorflow as tf
import networkx as nx

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'
data = 'round1_ijcai_18_train_20180301.txt'

target = 'is_trade'
features = ['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 
            'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
            'shop_review_num_level', 'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description']

sourcetarget = ['user_id', 'item_id']

nodecol = 'user_id'

In [ ]:
data = pd.read_csv(wd+data, sep=' ')

In [26]:
data = data.head(2000)

In [27]:
edgelist = pd.DataFrame()
edgelist['user_id'] = data.user_id
edgelist['item_id'] = data.item_id

df_target = data[sourcetarget+[target]].groupby(sourcetarget ,as_index=False).sum()
df_features = data[sourcetarget+features].groupby(sourcetarget, as_index=False).mean()

df = df_target.merge(df_features, how='left', on=sourcetarget)

In [50]:
def onehot_encoder(target_ls):
    classes = set(target_ls)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    target_onehot = np.array(list(map(classes_dict.get, target_ls)), dtype=np.int32)
    return target_onehot, [*classes]

def adjacency_normalizer(adj):
    adj = adj + np.eye(adj.shape[0])
    d = np.diagflat(np.power(np.array(adj.sum(1)), -1))
    a_norm = d.dot(adj)
    return a_norm

def graph_convolution(a, x, w, b):
    with tf.name_scope('gcn_layer'):
        kernel = tf.add(tf.matmul(a, tf.matmul(x, w)), b)
        return tf.nn.leaky_relu(kernel)

In [51]:
g = nx.from_pandas_edgelist(edgelist, 
                            source=edgelist.columns[0], 
                            target=edgelist.columns[1], 
                            create_using=nx.MultiGraph())

a = adjacency_normalizer(nx.to_numpy_matrix(g))

In [52]:
df_node = pd.DataFrame({nodecol: g.nodes()})
df_node = df_node.join(df, how='left', on=nodecol, lsuffix='l_', rsuffix='r_')

x = df_node[features]
y, classes = onehot_encoder(list(df_node[target]))

In [53]:
n_nodes = g.number_of_nodes()
n_featues = len(features)
n_classes = len(classes)

In [54]:
keep_prob = 0.5
learning_rate = 0.001

n_epochs = 100
verbose = 10
loss_ls = []

In [55]:
adjacency = tf.placeholder(tf.float32, shape=(n_nodes, n_nodes), name='adjacency')
feature = tf.placeholder(tf.float32, shape=(n_nodes, n_featues), name='feature')
label = tf.placeholder(tf.float32, shape=(n_nodes, n_classes), name='label')

weights = tf.Variable(tf.random_normal([n_featues, n_classes], seed=0), name='weights')
bais = tf.Variable(tf.random_normal([n_nodes, n_classes], seed=0), name='bais')
    
hidden1 = graph_convolution(a=adjacency, x=feature, w=weights, b=bais)
hidden2 = tf.nn.dropout(hidden1, keep_prob=keep_prob)
output = tf.nn.softmax(logits=hidden2, axis=n_classes)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [56]:
predictions = None

In [57]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        training_op.run(feed_dict={adjacency: a, feature: x, label: y})
        if verbose <= 0:
            pass
        elif epoch % verbose == 0:
            err = loss.eval(feed_dict={adjacency: a, feature: x, label: y})
            loss_ls.append(err)
            print(epoch, '\tloss: %.6f' %err)
    predictions = output.eval(feed_dict={adjacency: a, feature: x, label: y})

InvalidArgumentError: Requires start <= limit when delta > 0: 1748/1
	 [[Node: range_11 = Range[Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](range_11/start, Sub_8, range_11/delta)]]

Caused by op 'range_11', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start(